## Imports and boring stuff

In [0]:
from google.colab import drive
from IPython.display import clear_output
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/vk_ads')
clear_output()

In [0]:
!pip install catboost
clear_output()

In [0]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings("ignore")

## Competition metric

In [0]:
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
  return np.abs(np.log(
      (responses_column + epsilon)
      / (answers_column + epsilon)
  )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
  log_accuracy_ratio_mean = np.array(
      [
          get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
          get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
          get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
      ]
  ).mean()

  percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

  return percentage_error.round(
      decimals=2
  )

def cost(answers, responses, epsilon=0.005):
  log_accuracy_ratio_mean = np.array(
      [
          get_smoothed_log_mape_column_value(responses[0], answers[0], epsilon),
          get_smoothed_log_mape_column_value(responses[1], answers[1], epsilon),
          get_smoothed_log_mape_column_value(responses[2], answers[2], epsilon),
      ]
  ).mean()

  percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

  return percentage_error.round(
      decimals=2
  )

def cv_cost(est, X, y):
    return cost(est.predict(X), y)

## Load data

In [0]:
def load_tsv(fp):
  return pd.read_csv(fp, sep='\t')

In [0]:
his_df = load_tsv('data/history.tsv')
us_df = load_tsv('data/users.tsv').astype('category')
xval_df = load_tsv('data/validate.tsv')
yval_df = load_tsv('data/validate_answers.tsv')

## How we cross validate

In [0]:
def crossval(model, xval_df, yval_df, train_sizes=[0.7, 0.75, 0.8]):
    scores = []
    for ts in train_sizes:              
        mid_scores = []
        data_sorted = pd.concat([xval_df, yval_df], axis=1).sort_values(by='hour_start')
        x_cols, y_cols = xval_df.columns, yval_df.columns
        n_train_samples = int(len(data_sorted) * ts)

        train_df = data_sorted[x_cols].iloc[:n_train_samples]
        y_train = te(data_sorted[y_cols].iloc[:n_train_samples].values)

        test_df = data_sorted[x_cols].iloc[n_train_samples:]
        y_test = te(data_sorted[y_cols].iloc[n_train_samples:].values)

        av_his = his_df[his_df['hour'] < test_df['hour_start'].min()]

        X_train, cat_features = fe(train_df, av_his)
        X_test, _ = fe(test_df, av_his)
        
        model.fit(X_train, y_train)
        clear_output()
        mid_scores.append(cost(pe(y_test), pe(model.predict(X_test))))
        scores.append(mid_scores)
    print('\n'.join([('%f percenst split: '+str(scores[i])) % train_sizes[i] for i in range(len(scores))]))
    scores = np.array(scores)
    return (scores.mean(axis=0), scores.std(axis=0))

# Machine learning actually

In [0]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def get_ta(ad):  # return df of users
    ids = [int(i) for i in ad['user_ids'].split(',')]
    aus = ad['audience_size']
    ta = us_df[us_df['user_id'].isin(ids)]  # target auditory
    assert ta.shape[0] == aus
    return ta

def get_n_tcities(ta):  # return number of target cities
    return sum([1 for i in ta['city_id'].value_counts().values if i != 0])

def get_ages_mean_std(ta):  # return mean and std of ages distr without outliers
    ta = ta.astype('int')
    ages = ta[(ta['age'] >= 14) & (ta['age'] <= 80)]['age']
    if ages.empty:
        ages = np.array([0])
    return [ages.mean(), ages.std()]

def get_male_perc(ta):  # percentage of men
    return ta['sex'].value_counts(normalize=True)[1] * 100

def get_new_features(ad):
    ta = get_ta(ad)
    ta = ta.astype('int')
    new_cols = []
    new_cols.append(get_n_tcities(ta))
    new_cols += get_ages_mean_std(ta)
    new_cols.append(get_male_perc(ta))
    return new_cols

def users_hist_features(ad, hd_grouped):
    # returns:
    # 1. mean number of seen ads for target auditory
    return pd.Series(hd_grouped.loc[ad.users].agg(['mean']).values.flatten(), index=['mean_ads_seen_per_user',])

def pub_us_hist_features(ad, hist_grouped):
    ta = [int(i) for i in ad['user_ids'].split(',')]
    pubs = [int(i) for i in ad.publishers.split(',')]
    ta_tp_history = hist_grouped[((hist_grouped['publisher'].isin(pubs)) & (hist_grouped['user_id'].isin(ta)))]
    h = ta_tp_history.groupby('user_id').agg(['sum'])
    h.columns = ['publisher_size', 'n_seen_ads_on_theese_platforms']
    agg_funcs = ['median', 'mean', 'std', 'sum', percentile(0.25), percentile(0.75)]
    x = h['n_seen_ads_on_theese_platforms'].agg(agg_funcs)
    ### можно выбросить тех, кто ни разу не видел 
    x.index = [
               'ta_tp_seen_ads_median', 'ta_tp_seen_ads_mean', 'ta_tp_seen_ads_std', 'ta_tp_seen_ads_sum', 'ta_tp_seen_ads_q1', 'ta_tp_seen_ads_q3'
               ]
    x['n_of_people_who_didnt_see'] = len(ta) - len(h)
    x['n_of_people_who_saw_at_least_once'] = h['n_seen_ads_on_theese_platforms'].value_counts()[0:].sum()
    x['n_of_people_who_saw_at_least_twice'] = h['n_seen_ads_on_theese_platforms'].value_counts()[1:].sum()
    x['n_of_people_who_saw_at_least_three_times'] = h['n_seen_ads_on_theese_platforms'].value_counts()[2:].sum()
    return x

def fe(X, hist):  # feature engeneering, returns enged X, cat_features
    X['users'] = [list(map(int, i.split(','))) for i in X['user_ids']] 
    X['time_shown'] = X['hour_end'] - X['hour_start']
    hist['day_hour'] = hist['hour'] % 24
    new_X = pd.DataFrame()

    # basic ad features
    new_X['cpm'] = X['cpm']
    new_X['time_shown'] = X['time_shown']
    new_X['audience_size'] = X['audience_size']

    # user info features
    ui_X = X.apply(get_new_features, axis=1, result_type='expand')
    ui_cols = ['n_target_cities', 'tage_mean', 'tage_std', 'male_perc']
    ui_X.columns = ui_cols
    new_X = pd.concat([new_X, ui_X], axis=1)

    # history features 1
    hist_grouped = hist.groupby('user_id')[['cpm']].agg(['size'])
    us_hist_X = X.apply(
        users_hist_features, axis=1, result_type='expand', args=(hist_grouped,),
    )
    new_X = pd.concat([new_X, us_hist_X], axis=1)

    # history features 2
    hist_grouped = hist.groupby(['publisher', 'user_id'])['cpm'].agg(['size']).reset_index()
    pub_us_hist_X = X.apply(
        pub_us_hist_features, axis=1, result_type='expand', args=(hist_grouped,)
    )
    new_X = pd.concat([new_X, pub_us_hist_X], axis=1)

    # define categorical features
    cat_features = []
    new_X[cat_features] = new_X[cat_features].astype('int')
    poly = PolynomialFeatures(2)
    new_X = poly.fit_transform(new_X)
    return (new_X, cat_features)

def pe(y):  # postprocess target
    return y ** 2

def te(y):  # target engeneering
    return np.sqrt(y)

In [0]:
%%time
X, cat_features = fe(xval_df, his_df)
y = te(yval_df.values) 

## Define base models

In [0]:
class MyCatboostRegressor(CatBoostRegressor):
    def predict(self, data):
        preds = super(MyCatboostRegressor, self).predict(data)
        preds = np.maximum(preds, 0.)
        preds = np.minimum(preds, 1.)
        preds = np.round(preds, 4)
        return preds


class WorldGreatestModel(object):
    # basically, simple ensemble
    def __init__(self, estimators=None):
        self.estimators = estimators
    
    def fit(self, X, y, n_folds=4):
        for i in range(len(self.estimators)):
            self.estimators[i].fit(X, y)

    def predict(self, X):
        preds = [self.postprocess_y(self.estimators[i].predict(X)) for i in range(len(self.estimators))]
        preds = sum(preds) / len(self.estimators)
        return preds

    def postprocess_y(self, preds):
        preds = np.maximum(preds, 0.)
        preds = np.minimum(preds, 1.)
        preds = np.round(preds, 4)
        return preds

    def save_models(self, fps):
        assert len(fps) == len(self.estimators)
        for i in range(len(fps)):
            self.estimators[i].save_model(fps[i], 'json')

    def load_models(self, fps):
        self.estimators = [MyCatboostRegressor().load_model(fp, 'json') for fp in fps]
        return self

n_iterations = 3000

estimators = [
    MyCatboostRegressor(
      loss_function= 'MultiRMSE',
      iterations= n_iterations,
      random_seed=2,
    ),
    MyCatboostRegressor(
      loss_function= 'MultiRMSE',
      iterations=n_iterations,
      random_seed=189,
    ),
    MyCatboostRegressor(
      loss_function= 'MultiRMSE',
      iterations=n_iterations,
      random_seed=101,
    ),
    MyCatboostRegressor(
      loss_function= 'MultiRMSE',
      iterations=n_iterations,
      random_seed=42,
    ),
]

model = WorldGreatestModel(estimators)

In [0]:
if False:
    ensemble_size = 4
    file_names = ['tuned_ensemble/' + 'model' + str(i) for i in range(ensemble_size)]
    model.fit(X, y)
    model.save_models(file_names) 

## CV

In [0]:
%%time
if True:
    kfold5 = [0.5, 0.6, 0.7, 0.8, 0.9]
    kfold9 = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

    scores_mean, scores_std = crossval(
        model, xval_df, yval_df, train_sizes=kfold9 
    )
    print('Mean scores', scores_mean)
    print('Std of scores', scores_std)